<a href="https://colab.research.google.com/github/adelyaahmadullina/neuron_network/blob/main/%D0%9D%D0%B5%D0%B9%D1%80_%D1%81%D0%B5%D1%82%D0%B8_%D0%90%D1%85%D0%BC%D0%B0%D0%B4%D1%83%D0%BB%D0%BB%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создание нейросетевой модели для автоматической обработки естественного языка с использованием методов attention

In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Attention, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Загрузка датасета
df = pd.read_csv("/content/IMDB Dataset.csv")  # Заменить на фактическое имя файла

# Преобразование текстовых меток в числовые
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Параметры токенизации
vocab_size = 10000
max_sequence_length = 200

# Токенизация текста
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Разделение на обучающий и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(
    padded_sequences, df['sentiment'], test_size=0.2, random_state=42
)

# Создание модели
inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=256)(inputs)
lstm_layer = LSTM(512, return_sequences=True)(embedding_layer)
attention = Attention()([lstm_layer, lstm_layer])
combined = tf.concat([lstm_layer, attention], axis=-1)
outputs = Dense(1, activation='sigmoid')(combined)

model = Model(inputs=inputs, outputs=outputs)

# Компиляция модели и обучение
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Оценка модели на тестовом наборе
eval_results = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {eval_results[1]*100:.2f}%")


ParserError: Error tokenizing data. C error: EOF inside string starting at row 6271